In [1]:
# 必要なライブラリをすべてインポート
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler # StandardScalerを追加
from sklearn.model_selection import train_test_split

# Google Colabにファイルをアップロード
# ファイルを選択してアップロードしてください
from google.colab import files
uploaded = files.upload()

# アップロードしたCSVファイルを読み込む
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

#----------------------------------------------------
# 訓練データ（train_df）の前処理
#----------------------------------------------------

# Cabin列の削除（欠損値が多いため）
train_df = train_df.drop('Cabin', axis=1)

# 欠損値の補完
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].median())
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0])

# カテゴリカルデータの数値化
train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1})
train_df = pd.get_dummies(train_df, columns=['Embarked'], prefix='Embarked')

# 特徴量エンジニアリング
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
train_df['IsAlone'] = 0
train_df.loc[train_df['FamilySize'] == 1, 'IsAlone'] = 1


#----------------------------------------------------
# テストデータ（test_df）の前処理
# 訓練データと全く同じ処理を適用します
#----------------------------------------------------

# Cabin列の削除
test_df = test_df.drop('Cabin', axis=1)

# 欠損値の補完
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].median())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].median())
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})
test_df = pd.get_dummies(test_df, columns=['Embarked'], prefix='Embarked')

# 特徴量エンジニアリング
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1
test_df['IsAlone'] = 0
test_df.loc[test_df['FamilySize'] == 1, 'IsAlone'] = 1


#----------------------------------------------------
# モデルの学習と予測
#----------------------------------------------------

# モデルの学習に使う特徴量を定義
# スケーリング対象の特徴量をリストアップ
scaling_features = ['Pclass', 'Age', 'Fare', 'FamilySize']
# その他の特徴量
other_features = ['Sex', 'Embarked_S', 'Embarked_C', 'Embarked_Q', 'IsAlone']

# 特徴量と目的変数に分割
X_train_raw = train_df[scaling_features]
X_test_raw = test_df[scaling_features]
y_train = train_df['Survived']

# StandardScalerのインスタンスを作成
scaler = StandardScaler()

# 訓練データの特徴量をスケーリング
# fit_transform()は、スケーリングのルールを学習し、データを変換します
X_train_scaled = scaler.fit_transform(X_train_raw)

# テストデータの特徴量をスケーリング
# transform()は、訓練データで学習したルールを使ってデータを変換します
X_test_scaled = scaler.transform(X_test_raw)

# スケーリングしたデータと、スケーリングしないデータを結合
X_train = np.hstack([X_train_scaled, train_df[other_features].values])
X_test = np.hstack([X_test_scaled, test_df[other_features].values])

# ロジスティック回帰モデルのインスタンスを作成・学習
# スケーリング後は、max_iter=1000にしなくても警告が出なくなるはずです
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# テストデータで予測を実行
predictions = model.predict(X_test)


#----------------------------------------------------
# 提出用ファイルの作成
#----------------------------------------------------

# PassengerIdと予測結果をデータフレームにまとめる
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions
})

# CSVファイルとして出力
submission.to_csv('submission.csv', index=False)

# 完了メッセージと最初の5行を表示
print("--- 処理がすべて完了しました ---")
print("提出用ファイル 'submission.csv' が作成されました。")
print("ファイルの最初の5行:")
print(submission.head())

Saving test.csv to test.csv
Saving train.csv to train.csv
--- 処理がすべて完了しました ---
提出用ファイル 'submission.csv' が作成されました。
ファイルの最初の5行:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
